**Назаров Дмитрий Алексеевич 214-322**

# Задание 4 Transfer Learning

## Задание

Решить вашу задачу классификации используя перенос обучения. **Transfer Learning** - это процесс дообучения на **новых данных** нейросети, которая была обучена до этого на других данных, например, ImageNet.

Определить задачу по классификации изображений основываясь на вашей теме диссертации. Если тема никак не связывается с изображениями, то придумать абстрактную задачу, например классификация радиодеталей.

В рамках задачи выбрать количество классов объектов. На каждый класс подобрать минимум по 30 изображений и распределить их на обучающую (train) и проверочную выборку (val).

Выбрать 2-3 уже обученные модели, дообучить их на ваших данных и оценить результат. Обученную модель используйте для извлечения признаков, то есть нужно "заморозить" все веса, кроме последних полносвязных слоев. Их мы создаем заново и в соответствии с вашим количеством классов.

## Источники

1. [Transfer learning for computer vision tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

In [6]:
import os
from tqdm.autonotebook import tqdm, trange

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
from torchvision.io import read_image
from torchvision.models import ResNet101_Weights, ResNet18_Weights
import copy

In [7]:
# Преобразование обучающих данных для расширения обучающей выборки и её нормализация
# Для валидационной (тестовой) выборки только нормализация

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(244),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(244),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir =  "./hymenoptera_data" # папка с данными. Если запускаете в Colab, нужно скопировать данные к себе в директорию и примонтировать диск. Если запускаете локально --- скачайте данные.

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# специальный класс для загрузки данных в виде батчей
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=2) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

use_gpu = torch.cuda.is_available()

## Извлечение признаков

[Список](https://pytorch.org/vision/stable/models.html#classification) предобученных моделей в Pytorch.

### Загрузка модели

In [8]:
num_classes = 2 # Количество классов в вашей задаче

# weights = ResNet18_Weights.DEFAULT
# model_conv = torchvision.models.resnet18(weights=weights) # Выбирите 2-3 модели из списка и дообучите их (pretrained=True)
weights = ResNet101_Weights.DEFAULT
model_conv = torchvision.models.resnet101(weights=weights)
for param in model_conv.parameters():
    param.requires_grad = False

# Параметры вновь созданных моделей по умолчанию имеют requires_grad=True
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Обратите внимание, что оптимизируются только параметры последнего слоя, в отличие от предыдущего.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Уменьшение LR в 0,1 раза каждые 7 эпох
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to C:\Users\Shrek/.cache\torch\hub\checkpoints\resnet101-cd907fc2.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

### Функция для обучения

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Обучение и оценка

In [10]:
model_ft = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=12)

Epoch 0/11
----------
train Loss: 0.6902 Acc: 0.5625
val Loss: 0.6236 Acc: 0.7500

Epoch 1/11
----------
train Loss: 0.6354 Acc: 0.6458
val Loss: 0.5920 Acc: 0.8125

Epoch 2/11
----------
train Loss: 0.5428 Acc: 0.8333
val Loss: 1.8966 Acc: 0.9688

Epoch 3/11
----------
train Loss: 0.4467 Acc: 0.9583
val Loss: 1.5407 Acc: 0.9688

Epoch 4/11
----------
train Loss: 0.3942 Acc: 0.9792
val Loss: 3.1633 Acc: 0.9688

Epoch 5/11
----------
train Loss: 0.4215 Acc: 0.8333
val Loss: 1.0058 Acc: 0.9688

Epoch 6/11
----------
train Loss: 0.3321 Acc: 0.9583
val Loss: 0.8982 Acc: 0.9688

Epoch 7/11
----------
train Loss: 0.3580 Acc: 0.8958
val Loss: 1.5141 Acc: 0.9688

Epoch 8/11
----------
train Loss: 0.3210 Acc: 0.9583
val Loss: 1.4064 Acc: 0.9688

Epoch 9/11
----------
train Loss: 0.3403 Acc: 0.8958
val Loss: 1.1137 Acc: 0.9688

Epoch 10/11
----------
train Loss: 0.2812 Acc: 0.9375
val Loss: 1.8335 Acc: 0.9688

Epoch 11/11
----------
train Loss: 0.3600 Acc: 0.8750
val Loss: 1.5065 Acc: 0.9688

Tr

## Анализ данных

Проанализировать данные и сделать выводы о возможных проблемах.

Изображения для дообучения: "болты" и "гайки"
1) Модель "resnet101": обучение 12 эпох, точность 0.968750
2) Модель "resnet18": обучение 25 эпох, точность 1

Модель с более меньшим кол-вом слоёв выполнила свою работу точнее. Такое поведение объяснятеся тем, что веса для этой модели были подобраны более подходящие.